In [ ]:
# !pip install keras_cv_attention_models

In [1]:
import evals
from tensorflow import keras
import tensorflow as tf
import IJB_evals
import matplotlib.pyplot as plt
import models
import keras_cv_attention_models
from keras_cv_attention_models import efficientnet

In [ ]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(tf.config.list_physical_devices('GPU'))

2 ways to load the model

In [ ]:
#Either
basic_model = keras.models.load_model('checkpoints/EfficientFaceV2S.h5', compile=False)

In [ ]:
#If the above did not work, then you need to build the model then load the weights. e.g.,
basic_model = efficientnet.EfficientNetV2S(input_shape=(112, 112, 3), num_classes=0, drop_connect_rate=0.2, pretrained="imagenet", first_strides=1)
basic_model = models.buildin_models(basic_model, dropout=0.2, emb_shape=512, output_layer='F', bn_momentum=0.9, bn_epsilon=1e-5, add_pointwise_conv=True, pointwise_conv_act="swish", scale=True, use_bias=False)

basic_model.load_weights('checkpoints/EfficientFaceV2S.h5')

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/lfw.bin', batch_size=256, flip=True, PCA_acc=True)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/lfw.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/vgg2_fp.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/cfp_ff.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/cfp_fp.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/calfw.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/cplfw.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/agedb_30.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
tt = IJB_evals.IJB_test(lambda imgs: basic_model((tf.cast(imgs, "float32") - 127.5) * 0.0078125).numpy(), data_path='path_to_IJB_Dataset/ijb-testsuite/ijb', subset='IJBB', batch_size=16)

In [ ]:
score = tt.run_model_test_single()

In [ ]:
IJB_evals.plot_roc_and_calculate_tpr([score], names=[basic_model.name + "_IJBB"], label=tt.label)

In [ ]:
tt = IJB_evals.IJB_test(lambda imgs: basic_model((tf.cast(imgs, "float32") - 127.5) * 0.0078125).numpy(), data_path='C:/Users/mohda/Downloads/ijb-testsuite/ijb', subset='IJBC', batch_size=16)

In [ ]:
score = tt.run_model_test_single()

In [ ]:
IJB_evals.plot_roc_and_calculate_tpr([score], names=[basic_model.name + "_IJBC"], label=tt.label)

Plot face quality distribution using norm value of feature

In [ ]:
cc = tf.norm(ee.embs, axis=1).numpy()
_ = plt.hist(cc, bins=512, alpha=0.5, label='agedb_30 quality')
cc = tf.norm(tt.embs, axis=1).numpy()
_ = plt.hist(cc, bins=512, alpha=0.5, label='IJBC quality')
plt.legend()
plt.tight_layout()